<a href="https://colab.research.google.com/github/Kor-ok/Galaxiad/blob/main/Woof.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 🏁Install
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
directory_path = "/content/woof"
if not os.path.exists(directory_path):
    os.mkdir(directory_path)

os.chdir(directory_path)

# install bark (make sure you have torch>=2 for much faster flash-attention)
!pip install git+https://github.com/suno-ai/bark.git

from IPython.display import Audio
import numpy as np
from bark.generation import (
    generate_text_semantic,
    preload_models,
)
from bark.api import semantic_to_waveform
from bark import SAMPLE_RATE, generate_audio, preload_models

preload_models()

!pip install nltk

import nltk
nltk.download("punkt")

from scipy.io import wavfile
import io
import re

  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-nxq4r8xo
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-nxq4r8xo
  Resolved https://github.com/suno-ai/bark.git to commit 61710e5265cbc2f5eb098e08c6bee5d23dfff3fd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 26.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 110.3 MB/s eta 0:00:00
     ━━

In [ ]:
# @title 🐶 T4 usage is best  { vertical-output: true, form-width: "33%", display-mode: "form" }
inputscript = "Earth is the 3rd Planet of the Sol System and Capital of the Galactic Imperium of Stellar Communities and of the United Roman Empire. The Entire Planet has been fully Urbanized. Earth has only one City, Roma Magna. Regardless of where you go, you are always in Rome. Most of it has Skyscrapers with up to 10Km height, multiple Gravitational Gardens and deep Underground networks. Earth has been re-engineered to have multiple layers of Living space with an underground City, Level City, Sky City and the Heavens. Most of the Residents of Earth are either Noble, Military or Diplomat with Most industrial labor being Automated or being done off planet.  Underground City can be described as low cost but that would be betraying the pricing in comparison to the Rest of the Empire. To live in the Underground on earth means you are Rich by Any other metric. Living there is not a horrible existence, Massive caves with Artificial skies and green have been created to give the Illusion of Old Earth. Level City, or aboveground is quite similar to Underground though Rain, Clouds and access to the Sun(in some cases) is definitely an Improvement. Main Benefits is quick Access to the Space Elevators and Grav Gardens. Living in the Level means you are quite Rich or have been an Earth Resident since birth, giving you access to all its Amenities. Sky City. Artificially created Above ground Levels. Think of it creating another Upper Crust though patchy and inconsistent, living on them means that you always have the sun, and more importantly, that you are so rich most other Planets would find it revolting. That being said, Most things on earth are free. The Cost of the Places are what really can cost you a lot of money. Though just having personal Property on earth, if sold would mean you can buy entire new world Continents. Finally the Heavens, They are an Artificial ring above Earth designed for the High Society, those who have power, wealth and Influence. Just getting on it is Difficult and requires a special Entry allowance. There is one \u201CBuilding\u201D that is far beyond the Scope of Comprehension. About 20% of the Entire Surface area is the Imperial Palace or Deam Palatium or Himmel's Palast. About 5% of these are Massive walls that are 100s of KM deep with a height up to 15 Km. Inside these Massive walls are four long structures meeting in the middle which houses the True Imperial Palace Building. Between these Four Wings are the last, from space visible, Green on Earth. It houses a massive zoo within one of the 4, another is just a big park. The Inner Sanctum, The Empyrion is Not much known about only that the Highest Officials and the Family may enter it. Surrounding the planet is the Interface, a Massive Equatorial ring attached to the planet with massive space Elevators and is the Commercial hub of the planet. Luna has been Terraformed into a cold but habitable world with roughly 100billion Population.   It's worth noting that while the lower class on Earth may be incredibly wealthy by the standards of other planets, they are still considered lower class on Earth itself. The cost of living on Earth, especially in the more desirable areas like the Underground City, is extremely high, and many people who live there have inherited their wealth or have made their fortune through successful business ventures. The upper class, including those who live in the Heavens, are even wealthier and hold a disproportionate amount of power and influence on the planet. While the lower class on Earth may be able to afford luxury items and experiences that would be unimaginable to the average person on other planets, they are still subject to the rules and regulations of the upper class and must follow the dictates of those in power." # @param {type:"string"}
language = "en" # @param ["de", "en", "es", "fr", "hi", "it", "ja", "ko", "pl", "pt", "ru", "tr", "zh"] {allow-input: true}
# @markdown I recommend Voice 3 in English
voice = 3 # @param {type:"slider", min:0, max:9, step:1}
# @markdown *Woooof!*\
# @markdown Low Temperature=🐢\
# @markdown High Temperature=🤪
GEN_TEMP = 0.8 # @param {type:"slider", min:0.4, max:1.1, step:0.001}
WAFFLE = 0.075 # @param {type:"slider", min:0.02, max:0.1, step:0.001}

# Ensure voice is within the valid range (0-9)
if voice < 0 or voice > 9:
    raise ValueError("Voice must be an integer between 0 and 9")

script_template = """
{inputscript}
""".replace("\n", " ").strip()

# Replace the {inputscript} placeholder with the actual user input
script = script_template.format(inputscript=inputscript)

#NLTK preprocessing FIGURE OUT THE [] custom pattern THING LATER https://www.nltk.org/howto/tokenize.html
custom_pattern = r'(?<!\[.*?)(?<!\])\s*[.!?](?=\s+|$)'
sentences = nltk.sent_tokenize(script)

# Construct the SPEAKER string
SPEAKER = f"v2/{language}_speaker_{voice}"

silence = np.zeros(int(0.25 * SAMPLE_RATE))  # quarter second of silence

# Count the number of characters in the user input
num_characters = len(inputscript)
# Calculate the result in minutes
result_in_minutes = round(num_characters * 0.3198 / 60)
print(f"\033[1m\033[38;5;208mUsing T4 this will take approx {result_in_minutes} minutes\033[0m")

#WOOF
pieces = []
for sentence in sentences:
    semantic_tokens = generate_text_semantic(
        sentence,
        history_prompt=SPEAKER,
        temp=GEN_TEMP,
        min_eos_p=WAFFLE,  # this controls how likely the generation is to end
    )

    audio_array = semantic_to_waveform(semantic_tokens, history_prompt=SPEAKER,)
    pieces += [audio_array, silence.copy()]

Audio(np.concatenate(pieces), rate=SAMPLE_RATE, autoplay=True)




In [ ]:
# @title Download
from google.colab import files

output_buffer = io.BytesIO()
wavfile.write(output_buffer, SAMPLE_RATE, np.concatenate(pieces))

output_buffer.seek(0)
output_data = output_buffer.read()
with open('audio.wav', 'wb') as f:
    f.write(output_data)

files.download('audio.wav')

In [ ]:
# @title Each Line of Text
# @markdown Maybe each line can be fed into image generation?
# Print each sentence on a new line
for sentence in sentences:
    print(sentence)

In [ ]:
# @title Need to figure out how to unload the Bark model from system RAM
import tensorflow as tf

# Clear GPU memory
tf.keras.backend.clear_session()


In [ ]:
# @title Install and use Speechbrain for audio enhancement
!pip install speechbrain
import torch
import torchaudio
from speechbrain.pretrained import SpectralMaskEnhancement

enhance_model = SpectralMaskEnhancement.from_hparams(
    source="/content/woof/speechbrain/metricgan-plus-voicebank",
    savedir="/content/woof/pretrained_models/metricgan-plus-voicebank",
)

# Load and add fake batch dimension
noisy = enhance_model.load_audio(
    "/content/woof/audio.wav"
).unsqueeze(0)

# Add relative length tensor
enhanced = enhance_model.enhance_batch(noisy, lengths=torch.tensor([1.]))

# Saving enhanced signal on disk
torchaudio.save('/content/woof/enhanced_audio.wav', enhanced.cpu(), 16000)



In [ ]:
# @title Download
from IPython.display import Audio

# Path to the audio file
audio_file_path = '/content/woof/enhanced_audio.wav'

# Display the audio player
Audio(audio_file_path, rate=16000, autoplay=True)


from google.colab import files

files.download('/content/woof/enhanced_audio.wav')

In [ ]:
# @title WebM
!pip install ffmpeg
!ffmpeg -i /content/woof/enhanced_audio.wav -vf "scale=300:100,setsar=1:1" -c:v libvpx -b:v 1M -c:a libvorbis /content/woof/enhanced_audio.webm
files.download('/content/woof/enhanced_audio.webm')


# Make a Long-Form Dialog with Bark

### Step 1: Format a script and speaker lookup

In [ ]:
speaker_lookup = {"Samantha": "v2/en_speaker_9", "John": "v2/en_speaker_2"}

# Script generated by chat GPT
script = """
Samantha: Hey, have you heard about this new text-to-audio model called "Bark"?

John: No, I haven't. What's so special about it?

Samantha: Well, apparently it's the most realistic and natural-sounding text-to-audio model out there right now. People are saying it sounds just like a real person speaking.

John: Wow, that sounds amazing. How does it work?

Samantha: I think it uses advanced machine learning algorithms to analyze and understand the nuances of human speech, and then replicates those nuances in its own speech output.

John: That's pretty impressive. Do you think it could be used for things like audiobooks or podcasts?

Samantha: Definitely! In fact, I heard that some publishers are already starting to use Bark to create audiobooks. And I bet it would be great for podcasts too.

John: I can imagine. It would be like having your own personal voiceover artist.

Samantha: Exactly! I think Bark is going to be a game-changer in the world of text-to-audio technology."""
script = script.strip().split("\n")
script = [s.strip() for s in script if s]
script

### Step 2: Generate the audio for every speaker turn

In [ ]:
pieces = []
silence = np.zeros(int(0.5*SAMPLE_RATE))
for line in script:
    speaker, text = line.split(": ")
    audio_array = generate_audio(text, history_prompt=speaker_lookup[speaker], )
    pieces += [audio_array, silence.copy()]

### Step 3: Concatenate all of the audio and play it

In [ ]:
Audio(np.concatenate(pieces), rate=SAMPLE_RATE)